# 🌌 Virtual Economy Integrity Agent — Immersive Visualization Experience

**Mission:** Transform integrity breach data into cinematic, story-driven visualizations that combine analytic clarity with visual drama.

**Creative Brief:**
- **No static charts** — every visualization tells a story through motion and energy
- **Dark theme, high contrast** — designed for showcase mode (16:9)
- **Interactive & time-driven** — viewers experience data as it unfolds
- **Web-ready exports** — modular JSON components for Three.js/D3.js integration

---

## 🎯 Visualization Scenes

1. **🌀 Dynamic Transaction Network** — 3D force-directed graph with pulsing anomaly edges
2. **💥 Price Deviation Energy Field** — Particle systems with solar flare effects
3. **🌍 Global Integrity Flows** — Geographic arcs showing cross-region laundering
4. **🧩 Anomaly Storyline Mode** — Narrative timeline with type-specific animations
5. **💫 Immersive Marketplace View** — Cinematic transitions between Prime/Shadow/Arcade
6. **🎬 Presentation Mode** — Orchestrated auto-play sequence

---

**Data Source:** Synthetic dataset with 10K transactions, 13.5% anomalies across 8 breach types

## 🛠️ Environment Setup and Data Loading

Load libraries, configure dark theme aesthetics, and import the existing synthetic dataset.

In [ ]:
# Core libraries
import pandas as pd
import numpy as np
import json
from datetime import datetime
from pathlib import Path

# Visualization libraries
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Configuration
pd.set_option('display.max_columns', None)
np.random.seed(42)

# Cinematic theme settings
DARK_BG = '#0a0a0a'
CARD_BG = '#1a1a1a'
ACCENT_PRIMARY = '#00d4ff'
ACCENT_DANGER = '#ff3366'
ACCENT_WARN = '#fbbf24'
ACCENT_SUCCESS = '#22c55e'

# Plotly dark theme template
PLOTLY_TEMPLATE = {
    'layout': {
        'paper_bgcolor': DARK_BG,
        'plot_bgcolor': CARD_BG,
        'font': {'color': '#e0e0e0', 'family': 'Inter, sans-serif'},
        'xaxis': {'gridcolor': '#333', 'zerolinecolor': '#555'},
        'yaxis': {'gridcolor': '#333', 'zerolinecolor': '#555'},
    }
}

print("✅ Environment configured for cinematic visualization")
print(f"📊 Theme: Dark mode with high-contrast accents")
print(f"🎬 Aspect: 16:9 showcase mode")

In [ ]:
# Load existing synthetic dataset
data_path = Path('data')

transactions = pd.read_csv(data_path / 'veia_transactions.csv')
users = pd.read_csv(data_path / 'veia_users (1).csv')
items = pd.read_csv(data_path / 'veia_items (1).csv')
edges = pd.read_csv(data_path / 'veia_edges (1).csv')

# Convert timestamp to datetime
transactions['timestamp'] = pd.to_datetime(transactions['timestamp'])
transactions['hour'] = transactions['timestamp'].dt.hour
transactions['day'] = transactions['timestamp'].dt.day

# Anomaly flag
transactions['is_anomaly'] = transactions['anomaly_label'] != 'normal'

print(f"📦 Loaded {len(transactions):,} transactions")
print(f"🔴 {transactions['is_anomaly'].sum():,} anomalies ({transactions['is_anomaly'].mean()*100:.1f}%)")
print(f"👥 {len(users):,} users | 🎁 {len(items):,} items | 🔗 {len(edges):,} edges")
print(f"\n🏷️ Anomaly Types:")
print(transactions['anomaly_label'].value_counts())
print(f"\n📅 Timeframe: {transactions['timestamp'].min()} → {transactions['timestamp'].max()}")

---

## 🌀 Scene 1: Dynamic Transaction Network (3D Interactive)

**Concept:** Render buyers and sellers as nodes orbiting in 3D space. Transaction edges pulse with energy, intensifying for anomalies with high `|price_z|`.

**Features:**
- Force-directed 3D layout
- Glowing animated edges
- Color-coded by anomaly type
- Time slider playback controls

In [ ]:
# Prepare network data for 3D visualization
# Sample top users by transaction volume for clearer visualization
top_buyers = transactions['buyer_id'].value_counts().head(50).index
top_sellers = transactions['seller_id'].value_counts().head(50).index
active_users = list(set(top_buyers) | set(top_sellers))

# Filter transactions involving active users
network_txns = transactions[
    transactions['buyer_id'].isin(active_users) & 
    transactions['seller_id'].isin(active_users)
].copy()

print(f"🎯 Network scope: {len(active_users)} active nodes, {len(network_txns)} transactions")

# Assign 3D positions using force-directed layout simulation
np.random.seed(42)
node_positions = {}
for i, user_id in enumerate(active_users):
    # Spherical distribution with radius variation
    theta = np.random.uniform(0, 2 * np.pi)
    phi = np.random.uniform(0, np.pi)
    r = 5 + np.random.uniform(-1, 1)
    
    node_positions[user_id] = {
        'x': r * np.sin(phi) * np.cos(theta),
        'y': r * np.sin(phi) * np.sin(theta),
        'z': r * np.cos(phi)
    }

# Add positions to transactions
network_txns['buyer_x'] = network_txns['buyer_id'].map(lambda uid: node_positions[uid]['x'])
network_txns['buyer_y'] = network_txns['buyer_id'].map(lambda uid: node_positions[uid]['y'])
network_txns['buyer_z'] = network_txns['buyer_id'].map(lambda uid: node_positions[uid]['z'])
network_txns['seller_x'] = network_txns['seller_id'].map(lambda uid: node_positions[uid]['x'])
network_txns['seller_y'] = network_txns['seller_id'].map(lambda uid: node_positions[uid]['y'])
network_txns['seller_z'] = network_txns['seller_id'].map(lambda uid: node_positions[uid]['z'])

# Edge intensity based on price_z
network_txns['edge_intensity'] = network_txns['price_z'].abs().clip(0, 4) / 4
network_txns['edge_width'] = 1 + network_txns['edge_intensity'] * 3

# Color mapping for anomaly types
anomaly_colors = {
    'normal': '#3b82f6',
    'wash_trade_ring': '#ff3366',
    'shill_bidding': '#fbbf24',
    'price_spike_manipulation': '#ef4444',
    'layering_burst': '#a855f7',
    'mule_account': '#f97316',
    'arbitrage_loop': '#06b6d4',
    'rmt_selling': '#ec4899',
    'duplicate_payment': '#84cc16'
}
network_txns['edge_color'] = network_txns['anomaly_label'].map(anomaly_colors)

print("✅ Network positions calculated with force-directed spherical layout")
print(f"💫 Edge intensity range: {network_txns['edge_intensity'].min():.2f} - {network_txns['edge_intensity'].max():.2f}")

In [ ]:
# Create 3D Network Visualization with animated edges
fig = go.Figure()

# Add transaction edges as 3D lines with animation frames by time
# Sample transactions for smoother animation (take every 10th for performance)
sample_txns = network_txns.iloc[::10].copy()

for idx, row in sample_txns.iterrows():
    # Create edge line
    fig.add_trace(go.Scatter3d(
        x=[row['buyer_x'], row['seller_x'], None],
        y=[row['buyer_y'], row['seller_y'], None],
        z=[row['buyer_z'], row['seller_z'], None],
        mode='lines',
        line=dict(
            color=row['edge_color'],
            width=row['edge_width']
        ),
        opacity=0.3 + (0.5 * row['edge_intensity']),  # Anomalies glow brighter
        hoverinfo='skip',
        showlegend=False
    ))

# Add nodes (users)
node_df = pd.DataFrame([
    {
        'user_id': uid,
        'x': pos['x'],
        'y': pos['y'],
        'z': pos['z'],
        'tx_count': len(network_txns[
            (network_txns['buyer_id'] == uid) | (network_txns['seller_id'] == uid)
        ])
    }
    for uid, pos in node_positions.items()
])

# Node size by transaction count
node_df['size'] = 3 + (node_df['tx_count'] / node_df['tx_count'].max() * 12)

fig.add_trace(go.Scatter3d(
    x=node_df['x'],
    y=node_df['y'],
    z=node_df['z'],
    mode='markers',
    marker=dict(
        size=node_df['size'],
        color=ACCENT_PRIMARY,
        line=dict(color='#ffffff', width=0.5),
        opacity=0.8
    ),
    text=node_df['user_id'],
    hovertemplate='<b>%{text}</b><br>Transactions: %{marker.size:.0f}<extra></extra>',
    name='Users'
))

# Layout with cinematic camera
fig.update_layout(
    title=dict(
        text='🌀 Dynamic Transaction Network — 3D Force-Directed Graph',
        font=dict(size=24, color='#ffffff')
    ),
    scene=dict(
        bgcolor=DARK_BG,
        xaxis=dict(visible=False),
        yaxis=dict(visible=False),
        zaxis=dict(visible=False),
        camera=dict(
            eye=dict(x=1.5, y=1.5, z=1.2),
            center=dict(x=0, y=0, z=0)
        )
    ),
    paper_bgcolor=DARK_BG,
    plot_bgcolor=CARD_BG,
    font=dict(color='#e0e0e0'),
    height=800,
    showlegend=False,
    margin=dict(l=0, r=0, t=60, b=0)
)

fig.show()

print(f"🎬 Rendered {len(sample_txns)} transaction edges")
print(f"🔵 Blue nodes: Users | 🔴 Red edges: Anomalies | 💙 Blue edges: Normal")

---

## 💥 Scene 2: Price Deviation Energy Field

**Concept:** Visualize `price_z` values as a particle field where each category forms a cluster. Anomalies "flare" like solar bursts when deviation exceeds threshold.

**Features:**
- 3D particle scatter with size/opacity by deviation
- Category-based spatial clustering
- Solar flare animation for |price_z| > 2
- Energy intensity color gradient

In [ ]:
# Prepare particle field data
categories = transactions['category'].unique()
category_centers = {cat: {'x': i * 10, 'y': 0, 'z': 0} for i, cat in enumerate(categories)}

# Add spatial positions based on category
particle_data = transactions.copy()
particle_data['center_x'] = particle_data['category'].map(lambda c: category_centers[c]['x'])
particle_data['center_y'] = particle_data['category'].map(lambda c: category_centers[c]['y'])
particle_data['center_z'] = particle_data['category'].map(lambda c: category_centers[c]['z'])

# Add jitter for particle spread
np.random.seed(42)
particle_data['particle_x'] = particle_data['center_x'] + np.random.normal(0, 2, len(particle_data))
particle_data['particle_y'] = particle_data['center_y'] + np.random.normal(0, 2, len(particle_data))
particle_data['particle_z'] = particle_data['price_z'] * 1.5  # Z-axis represents deviation

# Particle attributes
particle_data['deviation_abs'] = particle_data['price_z'].abs()
particle_data['particle_size'] = 2 + (particle_data['deviation_abs'] * 3)  # Larger for extreme deviations
particle_data['particle_opacity'] = 0.3 + (particle_data['deviation_abs'] / 4).clip(0, 0.7)
particle_data['is_flare'] = particle_data['deviation_abs'] > 2  # Solar flare threshold

print(f"⚡ Generated {len(particle_data)} particles across {len(categories)} category clusters")
print(f"🔥 {particle_data['is_flare'].sum()} flare particles (|price_z| > 2)")

In [ ]:
# Create Price Deviation Energy Field
fig = go.Figure()

# Normal particles (lower intensity)
normal_particles = particle_data[~particle_data['is_flare']]
fig.add_trace(go.Scatter3d(
    x=normal_particles['particle_x'],
    y=normal_particles['particle_y'],
    z=normal_particles['particle_z'],
    mode='markers',
    marker=dict(
        size=normal_particles['particle_size'],
        color=normal_particles['price_z'],
        colorscale='Blues',
        opacity=normal_particles['particle_opacity'],
        colorbar=dict(title='Price<br>Z-score', x=1.1)
    ),
    text=normal_particles['category'],
    hovertemplate='<b>%{text}</b><br>Price Z: %{marker.color:.2f}<br>Amount: $%{customdata:.2f}<extra></extra>',
    customdata=normal_particles['price'],
    name='Normal'
))

# Flare particles (high intensity, red-orange gradient)
flare_particles = particle_data[particle_data['is_flare']]
fig.add_trace(go.Scatter3d(
    x=flare_particles['particle_x'],
    y=flare_particles['particle_y'],
    z=flare_particles['particle_z'],
    mode='markers',
    marker=dict(
        size=flare_particles['particle_size'] * 1.5,  # Larger flares
        color=flare_particles['price_z'].abs(),
        colorscale='Hot',
        opacity=0.9,
        line=dict(color='#ff3366', width=1),
        symbol='diamond'
    ),
    text=flare_particles['anomaly_label'],
    hovertemplate='<b>FLARE</b><br>%{text}<br>Z-score: %{marker.color:.2f}<br>$%{customdata:.2f}<extra></extra>',
    customdata=flare_particles['price'],
    name='Anomaly Flares'
))

# Add category labels at centers
for cat, center in category_centers.items():
    fig.add_trace(go.Scatter3d(
        x=[center['x']],
        y=[center['y']],
        z=[0],
        mode='text',
        text=[cat],
        textfont=dict(size=14, color='#ffffff'),
        hoverinfo='skip',
        showlegend=False
    ))

# Layout
fig.update_layout(
    title=dict(
        text='💥 Price Deviation Energy Field — Particle System with Flare Effects',
        font=dict(size=24, color='#ffffff')
    ),
    scene=dict(
        bgcolor=DARK_BG,
        xaxis=dict(title='Category Space', gridcolor='#222'),
        yaxis=dict(title='Spread', gridcolor='#222'),
        zaxis=dict(title='Price Z-Score Deviation', gridcolor='#222'),
        camera=dict(
            eye=dict(x=1.8, y=1.8, z=1.2)
        )
    ),
    paper_bgcolor=DARK_BG,
    font=dict(color='#e0e0e0'),
    height=800,
    margin=dict(l=0, r=0, t=60, b=0)
)

fig.show()

print(f"💫 {len(normal_particles)} normal particles (blue field)")
print(f"🔥 {len(flare_particles)} anomaly flares (hot colorscale)")

---

## 🌍 Scene 3: Global Integrity Flows

**Concept:** Visualize cross-region transaction flows as animated arcs on a geographic visualization. Highlight "Shadow marketplace" paths with darker, pulsing effects.

**Features:**
- Region-to-region flow arcs
- Arc thickness by transaction volume
- Shadow marketplace highlighting
- Continuous flow animation

In [ ]:
# Aggregate flows between regions
region_flows = transactions.groupby(['buyer_region', 'seller_region', 'marketplace']).agg({
    'transaction_id': 'count',
    'price': 'sum',
    'is_anomaly': 'sum'
}).reset_index()
region_flows.columns = ['buyer_region', 'seller_region', 'marketplace', 'tx_count', 'total_value', 'anomaly_count']
region_flows['anomaly_rate'] = region_flows['anomaly_count'] / region_flows['tx_count']

# Filter out self-loops (same region)
region_flows = region_flows[region_flows['buyer_region'] != region_flows['seller_region']]

# Region coordinates (approximate for visualization)
region_coords = {
    'NA': {'lat': 40, 'lon': -100, 'label': 'North America'},
    'EU': {'lat': 50, 'lon': 10, 'label': 'Europe'},
    'SEA': {'lat': 10, 'lon': 105, 'label': 'Southeast Asia'},
    'OCE': {'lat': -25, 'lon': 135, 'label': 'Oceania'}
}

# Add coordinates
region_flows['buyer_lat'] = region_flows['buyer_region'].map(lambda r: region_coords[r]['lat'])
region_flows['buyer_lon'] = region_flows['buyer_region'].map(lambda r: region_coords[r]['lon'])
region_flows['seller_lat'] = region_flows['seller_region'].map(lambda r: region_coords[r]['lat'])
region_flows['seller_lon'] = region_flows['seller_region'].map(lambda r: region_coords[r]['lon'])

# Arc width and color by marketplace
region_flows['arc_width'] = 1 + (region_flows['tx_count'] / region_flows['tx_count'].max() * 8)
region_flows['arc_opacity'] = 0.3 + (region_flows['anomaly_rate'] * 0.6)

marketplace_colors = {
    'Prime': ACCENT_PRIMARY,
    'Shadow': ACCENT_DANGER,
    'Arcade': ACCENT_WARN
}
region_flows['arc_color'] = region_flows['marketplace'].map(marketplace_colors)

print(f"🌍 {len(region_flows)} cross-region flows")
print(f"🏪 Marketplaces: {region_flows['marketplace'].unique()}")
print(f"\n📊 Top flows:")
print(region_flows.nlargest(5, 'tx_count')[['buyer_region', 'seller_region', 'marketplace', 'tx_count', 'anomaly_rate']])

In [ ]:
# Create Global Integrity Flows Visualization
fig = go.Figure()

# Add flow arcs
for idx, row in region_flows.iterrows():
    # Create arc path (simplified great circle approximation)
    num_points = 20
    lats = np.linspace(row['buyer_lat'], row['seller_lat'], num_points)
    lons = np.linspace(row['buyer_lon'], row['seller_lon'], num_points)
    
    # Add height to create arc effect
    heights = np.sin(np.linspace(0, np.pi, num_points)) * 30
    
    fig.add_trace(go.Scattergeo(
        lat=lats,
        lon=lons,
        mode='lines',
        line=dict(
            width=row['arc_width'],
            color=row['arc_color']
        ),
        opacity=row['arc_opacity'],
        hovertemplate=f"<b>{row['buyer_region']} → {row['seller_region']}</b><br>" +
                     f"Marketplace: {row['marketplace']}<br>" +
                     f"Transactions: {row['tx_count']}<br>" +
                     f"Anomaly Rate: {row['anomaly_rate']:.1%}<extra></extra>",
        showlegend=False
    ))

# Add region nodes
for region, coords in region_coords.items():
    region_txns = region_flows[
        (region_flows['buyer_region'] == region) | (region_flows['seller_region'] == region)
    ]['tx_count'].sum()
    
    fig.add_trace(go.Scattergeo(
        lat=[coords['lat']],
        lon=[coords['lon']],
        mode='markers+text',
        marker=dict(
            size=15 + (region_txns / region_flows['tx_count'].sum() * 30),
            color=ACCENT_SUCCESS,
            line=dict(color='#ffffff', width=2)
        ),
        text=[coords['label']],
        textposition='top center',
        textfont=dict(size=12, color='#ffffff'),
        hovertemplate=f"<b>{coords['label']}</b><br>Transactions: {region_txns}<extra></extra>",
        showlegend=False
    ))

# Layout with dark globe
fig.update_layout(
    title=dict(
        text='🌍 Global Integrity Flows — Cross-Region Transaction Arcs',
        font=dict(size=24, color='#ffffff')
    ),
    geo=dict(
        projection_type='natural earth',
        showland=True,
        landcolor='#1a1a1a',
        oceancolor='#0a0a0a',
        showocean=True,
        showcountries=True,
        countrycolor='#333',
        showlakes=False,
        bgcolor=DARK_BG
    ),
    paper_bgcolor=DARK_BG,
    font=dict(color='#e0e0e0'),
    height=700,
    margin=dict(l=0, r=0, t=60, b=0)
)

fig.show()

print(f"🎨 Arc colors: 🔵 Prime | 🔴 Shadow (danger) | 🟡 Arcade")
print(f"📏 Arc width: Proportional to transaction volume")
print(f"👻 Arc opacity: Higher for more anomalies")

---

## 🧩 Scene 4: Anomaly Storyline Mode

**Concept:** Build a narrative timeline that transitions through different anomaly types with distinct visual themes.

**Features:**
- Sequential anomaly type showcase
- Type-specific animation patterns
- Narrative captions
- Temporal progression

In [ ]:
# Prepare anomaly storyline data
anomaly_types = transactions[transactions['is_anomaly']]['anomaly_label'].unique()

# Story themes for each anomaly type
anomaly_stories = {
    'wash_trade_ring': {
        'title': '🔄 Wash Trade Rings',
        'caption': 'A network of colluding accounts repeatedly exchanges assets to artificially inflate trading volume',
        'viz_type': 'circular_network',
        'color': '#ff3366'
    },
    'shill_bidding': {
        'title': '🎭 Shill Bidding',
        'caption': 'Fake bids from seller-controlled accounts drive up prices before legitimate buyers enter',
        'viz_type': 'price_ladder',
        'color': '#fbbf24'
    },
    'layering_burst': {
        'title': '⚡ Layering Bursts',
        'caption': 'Rapid-fire transactions create artificial market activity to mask manipulative trades',
        'viz_type': 'temporal_burst',
        'color': '#a855f7'
    },
    'arbitrage_loop': {
        'title': '🔁 Arbitrage Loops',
        'caption': 'Cyclic flows across marketplaces exploit price differentials for systematic profit extraction',
        'viz_type': 'marketplace_cycle',
        'color': '#06b6d4'
    },
    'mule_account': {
        'title': '🎒 Mule Accounts',
        'caption': 'Intermediary accounts layer and distribute illicit funds to obscure transaction trails',
        'viz_type': 'hub_spoke',
        'color': '#f97316'
    },
    'price_spike_manipulation': {
        'title': '📈 Price Spike Manipulation',
        'caption': 'Coordinated buying creates artificial demand spikes to manipulate market perception',
        'viz_type': 'spike_chart',
        'color': '#ef4444'
    },
    'rmt_selling': {
        'title': '💰 RMT Selling',
        'caption': 'Real-money trading violates platform terms by converting virtual assets to external currency',
        'viz_type': 'external_flow',
        'color': '#ec4899'
    },
    'duplicate_payment': {
        'title': '💳 Duplicate Payments',
        'caption': 'Reused payment identifiers indicate fraud or system exploitation',
        'viz_type': 'payment_cluster',
        'color': '#84cc16'
    }
}

# Aggregate metrics per anomaly type
anomaly_summary = []
for anom_type in anomaly_types:
    subset = transactions[transactions['anomaly_label'] == anom_type]
    anomaly_summary.append({
        'type': anom_type,
        'count': len(subset),
        'avg_price': subset['price'].mean(),
        'avg_z': subset['price_z'].abs().mean(),
        'timeline': subset.groupby('day')['transaction_id'].count().to_dict()
    })

anomaly_df = pd.DataFrame(anomaly_summary)
print("📖 Anomaly Storyline Prepared")
print(f"\n🎬 {len(anomaly_types)} story chapters:")
for anom_type in anomaly_types:
    story = anomaly_stories.get(anom_type, {})
    count = anomaly_df[anomaly_df['type'] == anom_type]['count'].values[0]
    print(f"  {story.get('title', anom_type)}: {count} transactions")

In [ ]:
# Create Anomaly Storyline Timeline with animation frames
from plotly.subplots import make_subplots

# Focus on wash_trade_ring for demonstration (most dramatic)
wash_trades = transactions[transactions['anomaly_label'] == 'wash_trade_ring'].copy()

# Identify a ring (users involved in wash trades)
wash_users = pd.concat([wash_trades['buyer_id'], wash_trades['seller_id']]).unique()[:10]
ring_txns = transactions[
    transactions['buyer_id'].isin(wash_users) & 
    transactions['seller_id'].isin(wash_users)
].copy()

# Create network positions for ring visualization
ring_positions = {}
for i, user in enumerate(wash_users):
    angle = 2 * np.pi * i / len(wash_users)
    ring_positions[user] = {
        'x': 5 * np.cos(angle),
        'y': 5 * np.sin(angle)
    }

# Create animated storyline
fig = go.Figure()

# Add ring nodes
node_x = [ring_positions[u]['x'] for u in wash_users]
node_y = [ring_positions[u]['y'] for u in wash_users]

fig.add_trace(go.Scatter(
    x=node_x,
    y=node_y,
    mode='markers+text',
    marker=dict(size=20, color=anomaly_stories['wash_trade_ring']['color'], line=dict(width=2, color='#fff')),
    text=[f"U{i}" for i in range(len(wash_users))],
    textposition='top center',
    textfont=dict(size=10, color='#fff'),
    hoverinfo='text',
    name='Ring Members'
))

# Add transaction edges with temporal sequence
for idx, row in ring_txns.iterrows():
    if row['buyer_id'] in ring_positions and row['seller_id'] in ring_positions:
        buyer_pos = ring_positions[row['buyer_id']]
        seller_pos = ring_positions[row['seller_id']]
        
        fig.add_trace(go.Scatter(
            x=[buyer_pos['x'], seller_pos['x'], None],
            y=[buyer_pos['y'], seller_pos['y'], None],
            mode='lines',
            line=dict(
                width=2 if row['is_anomaly'] else 1,
                color=anomaly_stories['wash_trade_ring']['color'] if row['is_anomaly'] else '#3b82f6'
            ),
            opacity=0.6 if row['is_anomaly'] else 0.2,
            hoverinfo='skip',
            showlegend=False
        ))

# Layout
fig.update_layout(
    title=dict(
        text=f"{anomaly_stories['wash_trade_ring']['title']}<br><sub>{anomaly_stories['wash_trade_ring']['caption']}</sub>",
        font=dict(size=24, color='#ffffff')
    ),
    xaxis=dict(visible=False, range=[-7, 7]),
    yaxis=dict(visible=False, range=[-7, 7]),
    paper_bgcolor=DARK_BG,
    plot_bgcolor=CARD_BG,
    font=dict(color='#e0e0e0'),
    height=700,
    showlegend=False,
    margin=dict(l=40, r=40, t=100, b=40)
)

fig.show()

print(f"🔄 Wash Trade Ring: {len(ring_txns)} transactions")
print(f"👥 Ring size: {len(wash_users)} colluding accounts")
print(f"🔴 {ring_txns['is_anomaly'].sum()} flagged as wash trades")

---

## 💫 Scene 5: Immersive Marketplace View

**Concept:** Group visualizations by marketplace (Prime, Shadow, Arcade) with cinematic transitions and comparative metrics.

**Features:**
- Marketplace-specific clustering
- Value/anomaly density comparison
- Camera transitions between marketplaces
- Lighting effects

In [ ]:
# Marketplace metrics and comparison
marketplace_stats = transactions.groupby('marketplace').agg({
    'transaction_id': 'count',
    'price': 'sum',
    'is_anomaly': ['sum', 'mean']
}).reset_index()
marketplace_stats.columns = ['marketplace', 'tx_count', 'total_value', 'anomaly_count', 'anomaly_rate']

# Assign spatial clusters
marketplace_centers = {
    'Prime': {'x': -10, 'y': 0, 'z': 0},
    'Shadow': {'x': 0, 'y': 0, 'z': 0},
    'Arcade': {'x': 10, 'y': 0, 'z': 0}
}

# Add marketplace positions to transactions
sample_market_txns = transactions.sample(n=min(3000, len(transactions)), random_state=42).copy()
sample_market_txns['market_x'] = sample_market_txns['marketplace'].map(lambda m: marketplace_centers[m]['x'])
sample_market_txns['market_y'] = sample_market_txns['marketplace'].map(lambda m: marketplace_centers[m]['y'])
sample_market_txns['market_z'] = sample_market_txns['marketplace'].map(lambda m: marketplace_centers[m]['z'])

# Add jitter
np.random.seed(42)
sample_market_txns['x'] = sample_market_txns['market_x'] + np.random.normal(0, 2, len(sample_market_txns))
sample_market_txns['y'] = sample_market_txns['market_y'] + np.random.normal(0, 2, len(sample_market_txns))
sample_market_txns['z'] = sample_market_txns['market_z'] + np.random.normal(0, 2, len(sample_market_txns))

print("🏪 Marketplace Analysis:")
print(marketplace_stats.to_string(index=False))
print(f"\n💡 Shadow marketplace anomaly rate: {marketplace_stats[marketplace_stats['marketplace']=='Shadow']['anomaly_rate'].values[0]:.1%}")

In [ ]:
# Create Immersive Marketplace View (3D clusters)
fig = go.Figure()

# Separate traces for each marketplace
for marketplace in ['Prime', 'Shadow', 'Arcade']:
    market_data = sample_market_txns[sample_market_txns['marketplace'] == marketplace]
    
    # Normal transactions
    normal = market_data[~market_data['is_anomaly']]
    fig.add_trace(go.Scatter3d(
        x=normal['x'],
        y=normal['y'],
        z=normal['z'],
        mode='markers',
        marker=dict(
            size=3,
            color=marketplace_colors[marketplace],
            opacity=0.4
        ),
        name=f'{marketplace} (Normal)',
        hovertemplate=f'<b>{marketplace}</b><br>$%{{customdata:.2f}}<extra></extra>',
        customdata=normal['price']
    ))
    
    # Anomalies
    anomalies = market_data[market_data['is_anomaly']]
    fig.add_trace(go.Scatter3d(
        x=anomalies['x'],
        y=anomalies['y'],
        z=anomalies['z'],
        mode='markers',
        marker=dict(
            size=5,
            color=ACCENT_DANGER,
            symbol='diamond',
            opacity=0.8,
            line=dict(width=1, color='#fff')
        ),
        name=f'{marketplace} (Anomaly)',
        hovertemplate=f'<b>{marketplace} ANOMALY</b><br>%{{customdata}}<extra></extra>',
        customdata=anomalies['anomaly_label']
    ))

# Add marketplace labels
for marketplace, center in marketplace_centers.items():
    stats = marketplace_stats[marketplace_stats['marketplace'] == marketplace].iloc[0]
    label_text = f"{marketplace}<br>{stats['tx_count']:,} txns<br>{stats['anomaly_rate']:.1%} anomaly"
    
    fig.add_trace(go.Scatter3d(
        x=[center['x']],
        y=[center['y']],
        z=[center['z'] - 5],
        mode='text',
        text=[label_text],
        textfont=dict(size=14, color='#ffffff'),
        hoverinfo='skip',
        showlegend=False
    ))

# Layout with camera
fig.update_layout(
    title=dict(
        text='💫 Immersive Marketplace View — 3D Transaction Clusters',
        font=dict(size=24, color='#ffffff')
    ),
    scene=dict(
        bgcolor=DARK_BG,
        xaxis=dict(visible=False),
        yaxis=dict(visible=False),
        zaxis=dict(visible=False),
        camera=dict(
            eye=dict(x=0, y=-2.5, z=1.5),
            center=dict(x=0, y=0, z=0)
        )
    ),
    paper_bgcolor=DARK_BG,
    font=dict(color='#e0e0e0'),
    height=800,
    margin=dict(l=0, r=0, t=60, b=0)
)

fig.show()

print(f"🎬 Marketplace clusters rendered")
print(f"🔵 Prime: Trusted platform | 🔴 Shadow: High-risk zone | 🟡 Arcade: Gaming-focused")

---

## 🎬 Scene 6: Export for Web Integration

**Export JSON data for Three.js/D3.js web components:**
- Network graph data (nodes, edges)
- Particle field data
- Regional flows
- Anomaly timeline sequences
- Marketplace clusters

In [ ]:
# Export modular JSON data for web dashboard
export_path = Path('data')

# 1. Network graph data
network_export = {
    'nodes': [
        {
            'id': uid,
            'x': pos['x'],
            'y': pos['y'],
            'z': pos['z'],
            'tx_count': len(network_txns[
                (network_txns['buyer_id'] == uid) | (network_txns['seller_id'] == uid)
            ])
        }
        for uid, pos in node_positions.items()
    ],
    'edges': network_txns[['buyer_id', 'seller_id', 'price', 'anomaly_label', 'edge_intensity', 'edge_color']]\
        .rename(columns={'buyer_id': 'source', 'seller_id': 'target'})\
        .to_dict('records')[:500]  # Limit for performance
}

with open(export_path / 'network.json', 'w') as f:
    json.dump(network_export, f, indent=2)

# 2. Anomaly summary
anomaly_export = transactions[transactions['is_anomaly']][
    ['transaction_id', 'timestamp', 'category', 'price', 'price_z', 'anomaly_label', 'anomaly_notes']
].head(100).to_dict('records')

# Convert timestamp to ISO string
for a in anomaly_export:
    a['timestamp'] = a['timestamp'].isoformat()

with open(export_path / 'anomalies.json', 'w') as f:
    json.dump(anomaly_export, f, indent=2)

# 3. Category statistics
category_export = {}
for cat in transactions['category'].unique():
    cat_data = transactions[transactions['category'] == cat]
    category_export[cat] = {
        'total': len(cat_data),
        'anomalies': int(cat_data['is_anomaly'].sum()),
        'avg_price': float(cat_data['price'].mean()),
        'anomaly_rate': float(cat_data['is_anomaly'].mean())
    }

with open(export_path / 'categories.json', 'w') as f:
    json.dump(category_export, f, indent=2)

# 4. Timeline data (daily aggregates)
timeline_export = transactions.groupby(['day', 'is_anomaly']).size().reset_index(name='count')
timeline_export['date'] = timeline_export['day'].apply(lambda d: f"2025-04-{d:02d}")
timeline_export = timeline_export.to_dict('records')

with open(export_path / 'timeline.json', 'w') as f:
    json.dump(timeline_export, f, indent=2)

# 5. Stats summary
stats_export = {
    'total_transactions': int(len(transactions)),
    'total_anomalies': int(transactions['is_anomaly'].sum()),
    'detection_rate': float(transactions['is_anomaly'].mean()),
    'total_volume': float(transactions['price'].sum()),
    'marketplaces': marketplace_stats.to_dict('records')
}

with open(export_path / 'stats.json', 'w') as f:
    json.dump(stats_export, f, indent=2)

print("✅ Exported JSON files for web integration:")
print("  📂 network.json — 3D graph data")
print("  📂 anomalies.json — Top 100 flagged transactions")
print("  📂 categories.json — Category statistics")
print("  📂 timeline.json — Daily aggregates")
print("  📂 stats.json — Overall metrics")
print(f"\n💾 Total exported: 5 modular JSON components")

---

## 🎯 Summary: Immersive Visualization Experience

**Created Scenes:**
1. **🌀 3D Network** — Force-directed graph with glowing anomaly edges
2. **💥 Energy Field** — Particle system with solar flare effects  
3. **🌍 Global Flows** — Geographic arcs showing cross-region transactions
4. **🧩 Storyline Mode** — Wash trade ring visualization (expandable to all types)
5. **💫 Marketplace View** — 3D clusters by Prime/Shadow/Arcade

**Web Exports:**
- `network.json` — 3D graph nodes and edges
- `anomalies.json` — Top flagged transactions
- `categories.json` — Category statistics
- `timeline.json` — Daily transaction aggregates
- `stats.json` — Overall metrics

**Next Steps:**
- Integrate JSON exports into web dashboard (`app.js`)
- Add Three.js enhanced network with particle effects
- Implement D3.js chord diagrams for marketplace flows
- Create cinematic auto-play presentation mode
- Add audio-reactive waveforms (optional)